In [1]:
import pandas as pd

/Users/emilsataev/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/emilsataev/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
df = pd.read_csv('notebooks_meta.csv')
kernels = pd.read_csv('../Kernels.csv')
df_comps = pd.read_csv('df_new_filtered.csv')
df.head()

,Unnamed: 0.1,Unnamed: 0,CompetitionId,TeamName,PublicLeaderboardRank,PrivateLeaderboardRank,TeamId,UserId,RequestDate,CurrentKernelVersionId,code,CleanCode
0,1,1,17807,Women Power🇮🇱,2.0,1.0,4240714,109123,01/14/2020,28356656,"[""import numpy as np \nimport pandas as pd # d...","\nfor dirname, _, filenames in os.walk('/kaggl..."
1,4,4,17807,WOS,19.0,33.0,4441942,4547744,02/24/2020,29140873,['# This Python 3 environment comes with many ...,"\nfor dirname, _, filenames in os.walk('/kaggl..."
2,6,6,17807,GoinUp,48.0,62.0,4252521,1457617,01/26/2020,28710497,['import pandas as pd\nimport numpy as np\nimp...,"train = pd.read_csv(""../input/widsdatathon2020..."
3,7,7,17807,Black Z-580,74.0,105.0,4328971,1003741,02/02/2020,28985739,['# In this notebook I have used fancyimpute t...,"\nfor dirname, _, filenames in os.walk('/kaggl..."
4,8,8,17807,Saurabh7,79.0,107.0,4314446,366544,01/29/2020,34088724,['# This Python 3 environment comes with many ...,"\nfor dirname, _, filenames in os.walk('/kaggl..."


In [3]:
kr_versions = pd.read_csv('../KernelVersions.csv')
kernels.head()

,Id,AuthorUserId,CurrentKernelVersionId,ForkParentKernelVersionId,ForumTopicId,FirstKernelVersionId,CreationDate,EvaluationDate,MadePublicDate,IsProjectLanguageTemplate,CurrentUrlSlug,Medal,MedalAwardDate,TotalViews,TotalComments,TotalVotes
0,1,2505,205.0,NaN,NaN,1.0,03/25/2015 18:25:32,03/23/2018,03/25/2015,False,hello,NaN,NaN,386,0,0
1,2,3716,1748.0,NaN,26670.0,2.0,03/25/2015 18:31:07,04/16/2015,03/25/2015,False,rf-proximity,3.0,07/15/2016,9131,1,13
2,4,3716,41.0,NaN,NaN,9.0,03/25/2015 21:57:36,03/23/2018,03/25/2015,False,r-version,NaN,NaN,165,0,0
3,5,28963,19.0,NaN,NaN,13.0,03/25/2015 22:01:04,03/23/2018,03/25/2015,False,test1,NaN,NaN,140,0,0
4,6,3716,21.0,NaN,NaN,15.0,03/25/2015 22:19:00,03/23/2018,03/25/2015,False,are-icons-missing,NaN,NaN,161,0,0


In [4]:
df_submissions = pd.read_csv('../Submissions.csv')


In [5]:
df_users = pd.read_csv('../Users.csv')

In [6]:
# Select only the relevant columns from df
df_relevant = df[['CurrentKernelVersionId', 'CompetitionId', 'UserId', 'PrivateLeaderboardRank']]

# Select only the relevant columns from kernels
kernels_relevant = kernels[['CurrentKernelVersionId', 'TotalComments', 'TotalVotes', 'CurrentUrlSlug']]

# Merge df with kernels to get kernel-related columns
merged_df = df_relevant.merge(kernels_relevant, on='CurrentKernelVersionId', how='left')

# Select only the relevant columns from df_comps
df_comps_relevant = df_comps[['Id', 'Title']]

# Merge the result with df_comps to get the competition name
merged_df = merged_df.merge(df_comps_relevant, left_on='CompetitionId', right_on='Id', how='left')

# Select only the relevant columns from df_submissions
df_submissions_relevant = df_submissions[['SourceKernelVersionId', 'PublicScoreFullPrecision']]

# Merge the result with df_submissions to get the kaggle_score
merged_df = merged_df.merge(df_submissions_relevant, left_on='CurrentKernelVersionId', right_on='SourceKernelVersionId', how='left')

# Select only the relevant columns from df_users
df_users_relevant = df_users[['Id', 'UserName']]

# Merge the result with df_users to get the UserName for kernel_link
merged_df = merged_df.merge(df_users_relevant, left_on='UserId', right_on='Id', how='left')

# Format kernel_link
merged_df['kernel_link'] = '/' + merged_df['UserName'] + '/' + merged_df['CurrentUrlSlug']

# Select and rename the required columns
result_df = merged_df[['CurrentKernelVersionId', 'TotalComments', 'TotalVotes', 'kernel_link', 'Title', 'PublicScoreFullPrecision', 'PrivateLeaderboardRank']]
result_df.columns = ['kernel_id', 'kaggle_comments', 'kaggle_upvotes', 'kernel_link', 'comp_name', 'kaggle_score', 'rank']

# Set kernel_id as the index
result_df.set_index('kernel_id', inplace=True)

result_df

,kaggle_comments,kaggle_upvotes,kernel_link,comp_name,kaggle_score,rank
kernel_id,,,,,,
28356656,11,52,/danofer/wids-2020-starter-catboost-0-9045-lb,WiDS Datathon 2020,NaN,1.0
29140873,6,12,/teamanalitycs34/lgbm-baseline,WiDS Datathon 2020,0.911996,33.0
28710497,1,13,/ggxgboostgg/wids2020-lgbm-starter,WiDS Datathon 2020,0.908023,62.0
28985739,16,10,/krrai77/wids2020-fancyimpute-lgb,WiDS Datathon 2020,NaN,105.0
34088724,0,0,/saurabh7/predict-patient-survival,WiDS Datathon 2020,NaN,107.0
...,...,...,...,...,...,...
162732847,0,3,/sailikithd004/obesity-prediction-rf,Multi-Class Prediction of Obesity Risk,0.000000,3573.0
162732847,0,3,/sailikithd004/obesity-prediction-rf,Multi-Class Prediction of Obesity Risk,0.000000,3573.0
163169721,0,2,/jaisuryasundarbabu/basic-multi-classification...,Multi-Class Prediction of Obesity Risk,0.000000,3576.0


In [7]:
result_df.to_csv('kernels_meta2.csv')

In [24]:
df_relevant = df[['CurrentKernelVersionId', 'code', 'CleanCode']]

df_relevant.columns = ['kernel_id', 'code', 'clean_code']

df_relevant.set_index('kernel_id', inplace=True)

df_relevant
df_relevant.to_csv('code_blocks2.csv')

In [8]:
import ast

In [9]:
# Function to create a new dataframe with code blocks
def create_code_blocks_df(df):
    data = []
    code_blocks_index = 0
    for index, row in df.iterrows():
        kernel_id = row['CurrentKernelVersionId']
        code_blocks = ast.literal_eval(row['code'])
        for code_block_id, code_block in enumerate(code_blocks):
            data.append({
                'code_blocks_index': code_blocks_index, 
                'kernel_id': kernel_id, 
                'code_block_id': code_block_id, 
                'code_block': code_block
            })
            code_blocks_index += 1
    
    result_df = pd.DataFrame(data)
    result_df.set_index('code_blocks_index', inplace=True)
    return result_df

code_blocks_df = create_code_blocks_df(df)

In [11]:
len(code_blocks_df['kernel_id'].unique().tolist())

18648

In [12]:
code_blocks_df.to_csv('code_blocks2.csv')